In [1]:
import time
from selenium.webdriver import Firefox
import pandas as pd

In [2]:
driver = Firefox()

In [3]:
driver.get("https://www.fullertonfund.com/fund/fullerton-sgd-cash-fund-a/")

In [4]:
time.sleep(0.5)
driver.find_element(by="class name", value="button-close").click()

In [5]:
n_pages = int(
    driver.find_element("id", "fund-range-history_paginate")
    .find_elements("class name", "paginate_button")[-2]
    .text
)

In [6]:
rows = []

In [7]:
for i in range(n_pages - 1):
    rows.extend(driver.find_element("id", "fund-range-history").text.split("\n")[1:])
    driver.find_element("id", "fund-range-history_next").click()
rows.extend(driver.find_element("id", "fund-range-history").text.split("\n")[1:])

In [8]:
driver.quit()

In [9]:
df = pd.DataFrame(
    pd.Series(rows[::-1]).str.split().apply(lambda x: x[:2]).to_list()
).set_axis(["date", "NAV"], axis=1)

In [10]:
df["date"] = pd.to_datetime(df["date"], dayfirst=True)

In [11]:
df["NAV"] = df["NAV"].str.replace("$", "").astype(float)

In [12]:
df.set_index("date").to_csv("data/Fullerton SGD Cash Fund A.csv")